## Import Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
##### 只有在colab上需要這幾行
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
!ln -s "/content/gdrive/My Drive/space_cola" /WORK1
#####

## Pandas Train Data

In [ ]:
dataframe_train = pd.read_csv('/content/gdrive/My Drive/space_colab/datasets/train_data-2018-09-29-2019-09-29.csv')
dataframe_train = dataframe_train.fillna(method='ffill')
del dataframe_train['Date']
#del dataframe_train['VFINX']
dataframe_train.shape

## Pandas Test Data

In [ ]:
dataframe_pred = pd.read_csv('/content/gdrive/My Drive/space_colab/datasets/predict_data-2019-09-30-2020-09-30.csv')
dataframe_pred = dataframe_pred.fillna(method='ffill')
del dataframe_pred['Date']
tmp = np.array(dataframe_pred['VFINX'])
#del dataframe_pred['VFINX']
dataframe_pred.head()

## Independent and identically distributed (I.I.D.) Testing

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
sm.graphics.tsa.plot_acf(tmp, lags=40)
plt.show()

## 切割資料

In [ ]:
train_A_d = dataframe_train[:125]
train_B_d = dataframe_train[:-124]
test_A_d = dataframe_pred[:200]
test_B_d = dataframe_pred[:-200]
test_B_d

## PCA降維

In [ ]:
import numpy as np  
from sklearn.decomposition import PCA

pca = PCA(n_components=11)  

X = np.array(train_A_d) 
Y = np.array(train_B_d) 
Z = np.array(test_A_d)
W = np.array(test_B_d)

newData_0 = pca.fit_transform(X)
print(pca.explained_variance_ratio_) ## [1]
newData_0.shape

In [ ]:
newData_1 = pca.fit_transform(Y)
print(pca.explained_variance_ratio_)##[0] 
newData_1.shape

In [ ]:
newData_2 = pca.fit_transform(Z)
print(pca.explained_variance_ratio_) ##[0]
newData_2

In [ ]:
newData_3 = pca.fit_transform(W)
print(pca.explained_variance_ratio_) ##[0]
newData_3.shape#

In [ ]:
print(pca)
newData_0.shape
## 

## 獲得各主成分的方差

In [ ]:
print(pca.explained_variance_ratio_) 
newData_0.shape

PCA

## Normalize test

In [ ]:
from scipy import stats
print(stats.shapiro(newData_0[:,0]))

## Independent and identically distributed (I.I.D.) Testing

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
sm.graphics.tsa.plot_acf(newData_3[:,0], lags=40)
plt.show()

## Min Max Scaler(資料切割)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

## train data
scaler = MinMaxScaler()
scaler = scaler.fit(newData_0)
scaled_train_dataA = scaler.fit_transform(newData_0)
scaled_train_dataA = Variable(torch.tensor(scaled_train_dataA).type(torch.FloatTensor))

scalers = MinMaxScaler()
scalers = scalers.fit(newData_1)
scaled_train_dataB = scaler.fit_transform(newData_1)
scaled_train_dataB = Variable(torch.tensor(scaled_train_dataB).type(torch.FloatTensor))

## Test Data

scalersTestA = MinMaxScaler()
scalersTestA = scalersTestA.fit(newData_2)
scaled_test_dataA = scalersTestA.fit_transform(newData_2)
scaled_test_dataA = Variable(torch.tensor(scaled_test_dataA).type(torch.FloatTensor))

## 
scalersTestB = MinMaxScaler()
scalersTestB = scalersTestB.fit(newData_3)
scaled_test_dataB = scalersTestB.fit_transform(newData_3)
scaled_test_dataB = Variable(torch.tensor(scaled_test_dataB).type(torch.FloatTensor))

scaled_train_dataA[:,1].shape

## Linear Model

In [ ]:
import torch.nn as nn
losses = []
_losses = []

model = nn.Sequential(torch.nn.Linear(11,7),
            torch.nn.Linear(7,3),
            torch.nn.Linear(3,1))
optimizer = torch.optim.SGD(model.parameters(), lr=0.091)
loss_function = torch.nn.MSELoss()

train = scaled_train_dataA
train_b = scaled_train_dataB[:,0]

## train mode
model = model.train()

for epoch in range(0,100):
    prediction = model(train)
    loss = loss_function(prediction,train_b)
    _loss = loss_function(prediction,train[:,0])
    losses.append(loss)
    _losses.append(_loss)
    optimizer.zero_grad()
    #loss.backward()
    _loss.backward()
    
    optimizer.step()
    if epoch % 10 == 0:
        print(loss)


## Testing and Combine

In [ ]:
test = scaled_test_dataA.reshape(200,11)
test_b = scaled_test_dataB[:,0].reshape(54,1)
## eval mode
model = model.eval()


### 將預測資料和原始測試及資料合併前，將原始測試及選擇那行刪除(不然合併會變成超過原本維度)
A = np.array(scaled_test_dataA)
A = np.delete(A,0,1)
pred = model(test)## 預測的資料
B = np.array(pred.data)
#A[:,0] = np.array(pred.data).reshape(200,)
x_plot, = plt.plot(B)
y_plot, = plt.plot(test[:,0])
plt.show()
final = np.column_stack((A,B))
## 合併完成後，利用PCA和min-max將整組資料反轉回去
final = scalersTestA.inverse_transform(final)
final = pca.inverse_transform(final)

final.shape


## Show Loss

In [ ]:
x_plot, = plt.plot(losses)
y_plot, = plt.plot(_losses)
plt.legend([x_plot,y_plot],['val_loss','loss'])
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.show()

## Draw Data(Scaler)

In [ ]:
plt.figure(figsize=(12,12))
plt.title("pred Data")
x_plot, = plt.plot(final[:,18],final[:,20],'o')## 合併後資料第21column是vfinx
y_plot, = plt.plot(final[:,18],tmp[:200],'o')#### testBd為當初切測試資料，留下沒用到的
plt.legend([x_plot,y_plot],['pred','data'])
plt.xlabel('column 20 data')
plt.ylabel('VFINX  data')
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
x_plot, = plt.plot(final[:,20])
y_plot, = plt.plot(tmp[:200])
plt.legend([x_plot,y_plot],['pred','data'])
plt.xlabel('data seq')
plt.ylabel('[0]')
plt.show()

In [ ]:
#data1 = np.array(test_b)
#data2 = np.array(pred.data)
#print(data1.shape)
#for i in range(0,124):
  #data1[i] = data1[i] * (dataframe_pred['VFINX'].max() - scal.min_[10]) + scal.min_[10]
  #data2[i] = data2[i] * (dataframe_pred['VFINX'].max() - scal.min_[10]) + scal.min_[10]

#plt.figure(figsize=(15,12))
#x_plot, = plt.plot(data1,'magenta')
#y_plot, = plt.plot(data2, 'g--')
#plt.xlabel('data seq')
#plt.ylabel('vfinx')
#plt.legend([x_plot,y_plot],['data','pred'])
#plt.show()